<h1> Battle of Neighborhoods- Capstone Project<br><h3> Data Analysis using Machine Learning algorithms and Foursqaure API to choose best place or city to establish food chain business in North India.<br><h4> Created By: Prateek Vashishtha  <br>Start Date: 07 June 2019

<h3> Importing Required Modules

In [1]:
!conda install -c conda-forge folium=0.5.0 --yes
#!conda install seaborn
#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will

In [89]:
address = 'New Delhi, India'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New Delhi are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New Delhi are 28.6141793, 77.2022662.


In [90]:
# create map of New York using latitude and longitude values
map_delhi = folium.Map(location=[latitude, longitude], zoom_start=10)
map_delhi

In [91]:
import json

In [92]:
CLIENT_ID = 'KEW0QGAUCDX2EUWJSVB4JO45JB20GGDW3ZID1YXFDR2V3EMQ' # your Foursquare ID
CLIENT_SECRET = 'ETNUWSZXOMNIICV3IO2AKU5JTXOXBLMTPOEKKDB4GI1TFHFE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KEW0QGAUCDX2EUWJSVB4JO45JB20GGDW3ZID1YXFDR2V3EMQ
CLIENT_SECRET:ETNUWSZXOMNIICV3IO2AKU5JTXOXBLMTPOEKKDB4GI1TFHFE


In [94]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 50000 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=KEW0QGAUCDX2EUWJSVB4JO45JB20GGDW3ZID1YXFDR2V3EMQ&client_secret=ETNUWSZXOMNIICV3IO2AKU5JTXOXBLMTPOEKKDB4GI1TFHFE&v=20180605&ll=28.6141793,77.2022662&radius=50000&limit=100'

In [95]:
import requests
results = requests.get(url).json()
results

{'meta': {'code': 429,
  'errorDetail': 'Quota exceeded',
  'errorType': 'quota_exceeded',
  'requestId': '5cfb44b96a607149414d3806'},
 'response': {}}

In [96]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [97]:
from pandas.io.json import json_normalize

In [98]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

KeyError: 'groups'

In [73]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

40 venues were returned by Foursquare.


In [74]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [75]:
delhi_venues = getNearbyVenues(names=nearby_venues['name'],
                                   latitudes=nearby_venues['lat'],
                                   longitudes=nearby_venues['lng']
                                  )

Gole Market Saket


KeyError: 'groups'

In [76]:
delhi_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Imperial,28.625548,77.218664,The Imperial,28.625548,77.218664,Hotel
1,The Imperial,28.625548,77.218664,HOTEL SARAVANA BHAVAN,28.627041,77.219514,South Indian Restaurant
2,The Imperial,28.625548,77.218664,"The Square, Cafe Coffee Day",28.626640,77.219288,Café
3,The Imperial,28.625548,77.218664,1911,28.625531,77.218715,Mediterranean Restaurant
4,The Imperial,28.625548,77.218664,The Spice Route,28.625577,77.218065,Asian Restaurant


In [77]:
delhi_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Amar Jawan Jyoti | अमर जवान ज्योति (Amar Jawan Jyoti),7,7,7,7,7,7
Ambience Mall,47,47,47,47,47,47
Amour Bistro,11,11,11,11,11,11
Bengali Market | बंगाली मार्केट | বাংলা বাজার,15,15,15,15,15,15
Big Chill,47,47,47,47,47,47
Big Yellow Door,14,14,14,14,14,14
Blue Bar,32,32,32,32,32,32
Blue Tokai Coffee Roasters,10,10,10,10,10,10
Cafe Coffee Day,32,32,32,32,32,32


In [78]:
delhi_venues[delhi_venues['Venue Category']=='South Indian Restaurant']

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1,The Imperial,28.625548,77.218664,HOTEL SARAVANA BHAVAN,28.627041,77.219514,South Indian Restaurant
253,Connaught Place | कनॉट प्लेस (Connaught Place),28.632731,77.220018,HOTEL SARAVANA BHAVAN,28.632319,77.216445,South Indian Restaurant
379,Wenger's,28.633412,77.218292,HOTEL SARAVANA BHAVAN,28.632319,77.216445,South Indian Restaurant
450,HOTEL SARAVANA BHAVAN,28.627041,77.219514,HOTEL SARAVANA BHAVAN,28.627041,77.219514,South Indian Restaurant
540,Hauz Khas Village,28.554311,77.195140,Naivedyam,28.554987,77.195104,South Indian Restaurant
754,Starbucks Coffee,28.632011,77.217731,HOTEL SARAVANA BHAVAN,28.632319,77.216445,South Indian Restaurant
897,Kunzum Travel Cafe,28.553684,77.194368,Naivedyam,28.554987,77.195104,South Indian Restaurant
1047,HOTEL SARAVANA BHAVAN,28.632319,77.216445,HOTEL SARAVANA BHAVAN,28.632319,77.216445,South Indian Restaurant
1590,Imperfecto,28.554657,77.195092,Naivedyam,28.554987,77.195104,South Indian Restaurant
1673,Khan Chacha | खान चाचा | خان چاچا,28.634202,77.220780,HOTEL SARAVANA BHAVAN,28.632319,77.216445,South Indian Restaurant


In [79]:
address = 'Meerut, India'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Meerut are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Meerut are 28.9963296, 77.7061915.


In [80]:
map_meerut = folium.Map(location=[latitude, longitude], zoom_start=10)
map_meerut

In [81]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 50000 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=KEW0QGAUCDX2EUWJSVB4JO45JB20GGDW3ZID1YXFDR2V3EMQ&client_secret=ETNUWSZXOMNIICV3IO2AKU5JTXOXBLMTPOEKKDB4GI1TFHFE&v=20180605&ll=28.9963296,77.7061915&radius=50000&limit=100'

In [82]:
results = requests.get(url).json()
results

{'meta': {'code': 429,
  'errorDetail': 'Quota exceeded',
  'errorType': 'quota_exceeded',
  'requestId': '5cfb4470f594df57ef3e818d'},
 'response': {}}

In [83]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

KeyError: 'groups'

In [ ]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

In [ ]:
meerut_venues = getNearbyVenues(names=nearby_venues['name'],
                                   latitudes=nearby_venues['lat'],
                                   longitudes=nearby_venues['lng']
                                  )

In [ ]:
meerut_venues.head()

In [ ]:
meerut_venues.groupby('Neighborhood').count()

In [84]:
meerut_venues[meerut_venues['Venue Category']=='Indian Restaurant']

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
7,Subway,28.997552,77.700613,The Food Club - TFC,28.998527,77.697907,Indian Restaurant
9,Subway,28.997552,77.700613,Empress Court,28.998560,77.697538,Indian Restaurant
13,Big Bite,28.928402,77.641204,Recipe Restaurant - BGR,28.924149,77.641814,Indian Restaurant
16,Cheetal Grand,29.244041,77.724737,Haveli,29.244478,77.725801,Indian Restaurant
17,Cheetal Grand,29.244041,77.724737,Captain Cavalry,29.242516,77.722420,Indian Restaurant
24,Krishna Sagar Restaurant,28.676298,77.443061,Krishna Sagar Restaurant,28.676298,77.443061,Indian Restaurant
26,Krishna Sagar Restaurant,28.676298,77.443061,Bikanerwala,28.675353,77.442574,Indian Restaurant
31,Haldiram's,29.344159,77.715689,Haldiram's,29.344159,77.715689,Indian Restaurant
43,McDonald's,29.344351,77.715620,Haldiram's,29.344159,77.715689,Indian Restaurant
49,Bikanervala,29.048882,77.706768,Bikanervala,29.048882,77.706768,Indian Restaurant


In [85]:
meerut_detail_venues = getNearbyVenues(names=meerut_venues['Venue'],
                                   latitudes=meerut_venues['Venue Latitude'],
                                   longitudes=meerut_venues['Venue Longitude']
                                  )

Gole Market Saket


KeyError: 'groups'

In [ ]:
meerut_detail_venues.shape

In [ ]:
meerut_detail_venues.head()

In [86]:
meerut_detail_venues['Venue Category'].unique()

array(['Food Court', 'Sporting Goods Shop', 'Motorcycle Shop', 'Pharmacy',
       'Hotel', 'Indian Restaurant', 'Bookstore', 'Fried Chicken Joint',
       'Sandwich Place', 'Clothing Store', 'Market', 'Department Store',
       'Flea Market', "Women's Store", 'Restaurant', 'Resort',
       'Scenic Lookout', 'Recreation Center', 'Café',
       'Fast Food Restaurant', 'Pizza Place', 'Shopping Mall', 'Multiplex',
       'Cafeteria', 'Coffee Shop', 'Park', 'Burger Joint', 'Bakery',
       'Ice Cream Shop', 'Arcade', 'Art Gallery', 'Juice Bar', 'ATM',
       'Accessories Store', 'Taxi Stand', 'Kitchen Supply Store',
       'Chinese Restaurant', 'Gym / Fitness Center',
       'Southern / Soul Food Restaurant', 'Bus Station', 'Noodle House',
       'Business Service', 'IT Services', 'Hardware Store', 'Snack Place',
       'Candy Store', 'Rest Area', 'Boat or Ferry'], dtype=object)

In [87]:
meerut_detail_venues[meerut_detail_venues['Venue Category']=='Indian Restaurant']

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
11,Himalayan Drug Store,28.976125,77.717031,The yellow chilli,28.972838,77.716277,Indian Restaurant
18,Subway,28.997552,77.700613,The Food Club - TFC,28.998527,77.697907,Indian Restaurant
20,Subway,28.997552,77.700613,Empress Court,28.998560,77.697538,Indian Restaurant
27,Bindals,28.996357,77.702173,The Food Club - TFC,28.998527,77.697907,Indian Restaurant
29,Bombay Bazaar,28.999051,77.698772,Empress Court,28.998560,77.697538,Indian Restaurant
30,Bombay Bazaar,28.999051,77.698772,The Food Club - TFC,28.998527,77.697907,Indian Restaurant
35,The Food Club - TFC,28.998527,77.697907,Empress Court,28.998560,77.697538,Indian Restaurant
36,The Food Club - TFC,28.998527,77.697907,The Food Club - TFC,28.998527,77.697907,Indian Restaurant
43,Jolly Shopping Centre,28.995770,77.698181,Empress Court,28.998560,77.697538,Indian Restaurant
47,Empress Court,28.998560,77.697538,Empress Court,28.998560,77.697538,Indian Restaurant


In [88]:
delhi_detail_venues=getNearbyVenues(names=delhi_venues['Venue'],
                                   latitudes=delhi_venues['Venue Latitude'],
                                   longitudes=delhi_venues['Venue Longitude']
                                  )

The Imperial


KeyError: 'groups'

In [ ]:
delhi_detail_venues.shape

In [ ]:
delhi_detail_venues.head()

In [ ]:
delhi_detail_venues[delhi_detail_venues['Venue Category']=='Indian Restaurant']